# ``State`` 简介

脑动力学建模过程中经常会处理随时间变化的状态变量，例如神经元的膜电位`V`，发放率模型的发放率`r`等，``BrainState`` 提供了 ``State`` 数据结构，帮助用户更直观地定义和管理计算状态。本教程将详细介绍 ``State`` 的使用方法，通过本教程你将学会：
- ``State`` 对象的基本概念与基本用法。
- 如何创建 ``State`` 对象，以及``State``对象所包含子类的用法：``ShortTermState``, ``LongTermState``, ``HiddenState``, ``ParamState``。
- 如何使用 ``StateTraceStack``去追踪程序中使用到的 ``State`` 对象。

In [2]:
import jax.numpy as jnp

import brainstate 
import brainunit as u

## 1. ``State`` 对象的基本概念与基本用法
``State`` 是 ``BrainState`` 中的一个重要数据结构，用于封装模型中的状态变量，这些状态变量主要表示模型中会随时间变化的变量。``State`` 可以wrap任意的Python数据，比如整数、浮点数、数组、``jax.Array``等，以及封装在字典或者数组中的上述任意Python数据。区别于Python原生数据结构，``State`` 对象的数据在程序编译之后是可变的。假设用户需要定义一个状态数组，可以用以下方法定义一个 ``State`` 对象：

In [3]:
example = brainstate.State(jnp.ones(10))

example

State(
  value=Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)
)

进一步而言，``State`` 支持任意的 [PyTree](https://jax.readthedocs.io/en/latest/working-with-pytrees.html)，这意味着用户可以将任意的数据结构封装在 ``State`` 对象中，方便地进行状态管理和计算。

In [4]:
example2 = brainstate.State({'a': jnp.ones(3), 'b': jnp.zeros(4)})

example2

State(
  value={'a': Array([1., 1., 1.], dtype=float32), 'b': Array([0., 0., 0., 0.], dtype=float32)}
)

状态变量的更新是必不可少的操作，用户可以通过 ``State.value`` 属性来访问和修改这些数据。比如访问操作：

In [5]:
example.value

Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

可以发现返回的就是我们在一开始传入的数组，接下来我们可以通过赋值操作来修改这个数组：

In [6]:
example.value = brainstate.random.random(3)

example

State(
  value=Array([0.34898925, 0.8469951 , 0.46985328], dtype=float32)
)

通过输出可以发现，``State`` 对象的数据已经被修改。``State`` 对象的数据是可变的，这意味着用户可以在程序运行时随时修改状态变量的值。这种特性对于脑动力学建模非常重要，因为模型中的状态变量通常会随时间变化。
这样我们可以总结出``State`` 语法的核心特性包括：

- 所有需要改变的量都被封装在 ``State`` 对象中，方便用户追踪和调试模型状态。
- 其它没有被 ``State`` 封装的变量都是不可变的，在程序编译后不能再被修改。

用户需要格外注意，**没有被标记为状态变量的数据将被作为静态数据进行JIT编译，对静态数据的修改在 JIT 编译环境中是无效的**。此外，通过``value``方法修改``State``对象的数据是有条件的，需要满足赋值的数据类型和形状与原数据一致，否则会报错。

In [7]:
state = brainstate.ShortTermState(jnp.zeros((2, 3)))

with brainstate.check_state_value_tree():
    state.value = jnp.zeros((2, 3))

    try:
        state.value = (jnp.zeros((2, 3)), jnp.zeros((2, 3)))
    except Exception as e:
        print(e)

The given value PyTreeDef((*, *)) does not match with the origin tree structure PyTreeDef(*).


## 2. ``State`` 对象的子类
``BrainState`` 提供了多种 ``State`` 的子类，用户可以根据自己的需求选择合适的子类。下面我们将详细介绍 ``ShortTermState``, ``LongTermState``, ``HiddenState``, ``ParamState`` 这四种子类的用法。

> 需要注意的是，``State`` 的子类是为了更好地管理状态变量而设计的，实际上这些上述四种子类在代码实现层面没有区别，主要帮助用户区别模型中所用状态变量的类型，用户在实际使用中可以根据自己的需求选择合适的子类。

### 2.1 ``ShortTermState``
``ShortTermState`` 是 ``State`` 的子类，用于封装模型中的短期状态变量。``ShortTermState`` 对象的数据在每次模拟迭代中都会被更新。用户可以通过 ``ShortTermState`` 对象来定义模型中的短期状态变量，比如神经元记录上一次发放的时间`last_spike_time`等。这些状态变量主要记录了模型中的瞬时状态，随时间展开并没有产生直接的时间依赖性，所以被称为短期状态变量。

In [8]:
# Example
short_term_state = brainstate.ShortTermState(jnp.ones(5))

# In Neuron model, we can use ShortTermState to record the last spike time of neuron
t_last_spike = brainstate.ShortTermState((brainstate.nn.param(brainstate.nn.Constant(-1e7 * u.ms), sizes=(10,))))
t_last_spike

ShortTermState(
  value=ArrayImpl([-10000000., -10000000., -10000000., -10000000., -10000000.,
             -10000000., -10000000., -10000000., -10000000., -10000000.],
            dtype=float32) * msecond
)

### 2.2 ``LongTermState``
``LongTermState`` 是 ``State`` 的子类，用于封装模型中的长期状态变量。``LongTermState`` 对象的数据在每次模拟迭代中都会被更新，但是在模拟迭代之间会被保留，也就是说长期状态变量在时间迭代过程中保留了历史信息，之前的状态信息会影响到下一步的值。用户可以通过 ``LongTermState`` 对象来定义模型中的长期状态变量，比如在计算滑动平均时，可以把平均值和方差作为长期状态变量定义。

In [9]:
# Example
long_term_state = brainstate.LongTermState(jnp.ones(5))

# We can use LongTermState to record the running mean of a variable
running_mean = brainstate.LongTermState(jnp.zeros(5))
running_mean

LongTermState(
  value=Array([0., 0., 0., 0., 0.], dtype=float32)
)

### 2.3 ``HiddenState``
``HiddenState`` 是 ``State`` 的子类，用于封装模型中的隐藏状态变量。``HiddenState`` 对象的数据在每次模拟迭代中都会被更新，但是在模拟迭代之间会被保留，所以它与``LongTermState``的用法是一致的。用户可以通过 ``HiddenState`` 对象来定义模型中的隐藏状态变量，比如神经元膜电位`V`，突触电导`g`，突触后电流`I`等。

In [10]:
# Example
hidden_state = brainstate.HiddenState(jnp.ones(5))

# In Neuron model, we can use HiddenState to record the membrane potential of neuron
V = brainstate.HiddenState((brainstate.init.param(brainstate.nn.Constant(-70. * u.mV), sizes=(10,))))
V

HiddenState(
  value=ArrayImpl([-70., -70., -70., -70., -70., -70., -70., -70., -70., -70.],
            dtype=float32) * mvolt
)

### 2.4 ``ParamState``
``ParamState`` 是 ``State`` 的子类，用于封装模型中的训练参数。``ParamState`` 对象主要用于定义可训练的模型当中的参数，例如神经网络的权重`w`和偏置`b`等。用户可以通过 ``ParamState`` 对象来定义模型中的参数。

In [11]:
# Example
param_state = brainstate.ParamState(jnp.ones(5))

# In Neural Network model, we can use ParamState to define the weight of the network
weight = brainstate.ParamState((brainstate.init.param(brainstate.init.Constant(0.1), sizes=(10, 10), batch_size=2)))
weight

ParamState(
  value=Array([[[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]],
  
         [[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
          [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.

需要再次强调，用户如果不使用 ``ShortTermState``, ``LongTermState``, ``HiddenState``, ``ParamState`` 这四种子类，可以直接使用 ``State`` 对象。这四种子类的主要作用是帮助用户更好地管理模型中的状态变量，区分状态变量的类型。

## 3. ``StateTraceStack`` 的使用

``StateTraceStack`` 是 ``BrainState`` 中的一个重要工具，用于追踪程序中使用到的 ``State`` 对象。由于``BrainState``在程序编译中将所有的`State`作为中间变量进行处理，所以不会存在一个全局空间将所有的`State`存储起来，程序在函数中使用到的`State`将被管理存储起来，一旦函数结束执行就会被释放。这样一来，如何能得到在一段程序中使用了哪些`State`呢？``StateTraceStack``提供了非常好的解决办法。用户可以通过 ``StateTraceStack`` 来查看程序中使用到的 ``State`` 对象，方便用户对这些``State``进行集中处理。``StateTraceStack`` 可以作为一种context manager使用，下面我们举一个例子：假设我们定义了一个线性层，我们在执行该线性层的调用时使用``StateTraceStack``记录所使用的`State`信息，并通过不同的函数将所读出或修改的`State`信息进行输出。



In [20]:
class Linear(brainstate.graph.Node):
    def __init__(self, din: int, dout: int):
        self.din, self.dout = din, dout
        self.w = brainstate.ParamState(brainstate.random.rand(din, dout))
        self.b = brainstate.ParamState(jnp.zeros((dout,)))
        self.y = brainstate.HiddenState(jnp.zeros((dout,)))

    def __call__(self, x):
        self.y.value = x @ self.w.value + self.b.value
        return self.y.value
    
model = Linear(2, 5)

with brainstate.StateTraceStack() as stack:
    model(brainstate.random.rand(2))
    states_to_be_read = [st for st in stack.get_read_states()]
    states_values_to_be_read = [st for st in stack.get_read_state_values()] 
    states_to_be_written = [st for st in stack.get_write_states()]
    states_values_to_be_written = [st for st in stack.get_write_state_values()]

我们可以通过``StateTraceStack``的四个方法``get_read_states``, ``get_read_state_values``, ``get_write_states``, ``get_write_state_values``来获取程序中使用到的``State``对象。这样我们可以更好地追踪程序中使用到的``State``对象，方便我们对这些``State``对象进行集中处理。首先，``get_read_states``方法返回的是程序中读取的``State``对象：

In [21]:
states_to_be_read

[ParamState(
   value=Array([[0.13818443, 0.2319262 , 0.14350498, 0.70770407, 0.15762603],
          [0.49146724, 0.85079646, 0.9801016 , 0.5602509 , 0.9497303 ]],      dtype=float32)
 ),
 ParamState(
   value=Array([0., 0., 0., 0., 0.], dtype=float32)
 )]

接着，``get_read_state_values``方法返回的是程序中读取的``State``对象所存储的值：

In [22]:
states_values_to_be_read

[Array([[0.13818443, 0.2319262 , 0.14350498, 0.70770407, 0.15762603],
        [0.49146724, 0.85079646, 0.9801016 , 0.5602509 , 0.9497303 ]],      dtype=float32),
 Array([0., 0., 0., 0., 0.], dtype=float32)]

``get_write_states``方法返回的是程序中写入的``State``对象：

In [23]:
states_to_be_written

[RandomState([3268126397  777968873]),
 HiddenState(
   value=Array([0.5366626 , 0.9263857 , 1.0222216 , 0.81177783, 0.997295  ],      dtype=float32)
 )]

``get_write_state_values``方法返回的是程序中写入的``State``对象所存储的值：

In [24]:
states_values_to_be_written

[Array([3268126397,  777968873], dtype=uint32),
 Array([0.5366626 , 0.9263857 , 1.0222216 , 0.81177783, 0.997295  ],      dtype=float32)]